# Cluster based Summarization

In [6]:
import os
import codecs
import warnings
warnings.filterwarnings("ignore")

corpus_path = "news-corpus//"
article_paths = [os.path.join(corpus_path,p) for p in os.listdir(corpus_path)]

doc_complete = []
for path in article_paths:
    with open(path, 'rb') as f:
        doc_content = f.read().decode(errors='ignore')
        doc_complete.append(doc_content)


In [7]:
import re
for i in range(len(doc_complete)):
    doc_complete[i] = re.sub(r'[^\w\s.]', '', doc_complete[i])

In [8]:
if len(doc_complete) >= 2:
    doc_complete.pop(1)

In [9]:
len(doc_complete)

5

In [ ]:
#pip uninstall gensim

In [ ]:
#pip install summa

In [5]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
from summa.summarizer import summarize

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# Preprocess the text data

# if len(doc_complete) >= 2:
#     doc_complete.pop(1)

# Define stopwords and lemmatizer

stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess a document
def preprocess_document(document):
    # Split the document into sentences
    sentences = sent_tokenize(document)
    
    # Preprocess each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        # Tokenize the sentence
        tokens = word_tokenize(sentence.lower())
        
        # Remove stopwords and punctuation
        tokens = [token for token in tokens if token.isalpha() and token not in stopwords]
        
        # Lemmatize the tokens
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        preprocessed_sentences.append(tokens)
    
    return preprocessed_sentences

# Create a list of preprocessed documents
preprocessed_documents = [preprocess_document(doc) for doc in doc_complete]

# Flatten the list of tokens
flattened_documents = [" ".join(token for sentence in document for token in sentence) for document in preprocessed_documents]

# Vectorize the documents (TF-IDF)
vectorizer = TfidfVectorizer()
vectorized_docs = vectorizer.fit_transform(flattened_documents)

# Perform clustering (K-means)
num_clusters = 2 # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=40)
kmeans.fit(vectorized_docs)
cluster_labels = kmeans.labels_

# Generate summaries for each cluster
for cluster_id in range(num_clusters):
    cluster_docs = [flattened_documents[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    
    # Calculate the centroid of the cluster
    cluster_vectors = vectorized_docs[cluster_labels == cluster_id]
    centroid = cluster_vectors.mean(axis=0)
    
    # Calculate cosine distances between documents and the centroid
    distances = cosine_distances(centroid, cluster_vectors)[0]
    
    # Sort documents by distance
    sorted_indices = sorted(range(len(distances)), key=lambda i: distances[i])
    print(sorted_indices)
    
    # Select top documents for summary
    top_doc_indices = sorted_indices[:2]  # Adjust the number of documents for summary
    
    # Extract top sentences from the selected documents
    top_sentences = []
    for doc_index in top_doc_indices:
        sentences = sent_tokenize(cluster_docs[doc_index])
        top_sentences.extend(sentences[:5])  # Adjust the number of sentences per document
    
    # Generate summary 
    summary = ". ".join(top_sentences)
    summarized_text = summarize(summary, words=200)  # Adjust the number of words in the summary
    
    # Print the summary for the cluster
    print(f"Cluster {cluster_id + 1} Summary:")
    print(summarized_text)
    print("==========================")


/Users/kusumachalla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


[2, 0, 1]
Cluster 1 Summary:
union agriculture minister narendra singh tomar friday said government making effort boost production consumption millet speaking inaugurating millet experience centre mec dilli haat ina national capital cooperative nafed collaboration agriculture ministry established centre create awareness benefit millet encourage adoption among general public agriculture secretary manoj ahuja nafed md rajbir singh present event making effort enhance production yield processing consumption miller country tomar said minister highlighted multiple health benefit consuming millet nutritious tomar said millet climate resilient grown le water minimal use fertiliser pesticide said increase production millet also boost income farmer especially small marginal one tomar noted large number startup commendable job field millet rued importance millet plate food got reduced year acting upon india proposal supported country united nation general assembly declared year international year

/Users/kusumachalla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [7]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
from summa.summarizer import summarize

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# Preprocess the text data

# if len(doc_complete) >= 2:
#     doc_complete.pop(1)

# Define stopwords and lemmatizer
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess a document
def preprocess_document(document):
    # Split the document into sentences
    sentences = sent_tokenize(document)
    
    # Preprocess each sentence
    preprocessed_sentences = []
    for sentence in sentences:
        # Tokenize the sentence
        tokens = word_tokenize(sentence.lower())
        
        # Remove stopwords and punctuation
        tokens = [token for token in tokens if token.isalpha() and token not in stopwords]
        
        # Lemmatize the tokens
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
        
        preprocessed_sentences.append(tokens)
    
    return preprocessed_sentences

# Create a list of preprocessed documents
preprocessed_documents = [preprocess_document(doc) for doc in doc_complete]

# Flatten the list of tokens
flattened_documents = [" ".join(token for sentence in document for token in sentence) for document in preprocessed_documents]

# Vectorize the documents (TF-IDF)
vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.1)  # Adjust the values as needed
vectorized_docs = vectorizer.fit_transform(flattened_documents)

# Perform clustering (K-means)
num_clusters = 3  # Adjust the number of clusters as needed
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', random_state=42)
kmeans.fit(vectorized_docs)
cluster_labels = kmeans.labels_

# Generate summaries for each cluster
for cluster_id in range(num_clusters):
    cluster_docs = [flattened_documents[i] for i, label in enumerate(cluster_labels) if label == cluster_id]
    
    # Calculate the centroid of the cluster
    cluster_vectors = vectorized_docs[cluster_labels == cluster_id]
    centroid = cluster_vectors.mean(axis=0)
    
    # Calculate cosine distances between documents and the centroid
    distances = cosine_distances(centroid, cluster_vectors)[0]
    
    # Sort documents by distance
    sorted_indices = sorted(range(len(distances)), key=lambda i: distances[i])
    
    print(sorted_indices)
    
    # Select top documents for summary
    top_doc_indices = sorted_indices[:5]  # Adjust the number of documents for summary
    
    # Extract top sentences from the selected documents
    top_sentences = []
    for doc_index in top_doc_indices:
        sentences = sent_tokenize(cluster_docs[doc_index])
        top_sentences.extend(sentences[:3])  # Adjust the number of sentences per document
    
    # Generate summary
    summary = ". ".join(top_sentences)
    summarized_text = summarize(summary, words=200)  # Adjust the number of words in the summary
    
    # Print the summary for the cluster
    print(f"Cluster {cluster_id + 1} Summary:")
    print(summarized_text)
    print("==========================")


[0]
Cluster 1 Summary:

[2, 0, 1]
Cluster 2 Summary:
union agriculture minister narendra singh tomar friday said government making effort boost production consumption millet speaking inaugurating millet experience centre mec dilli haat ina national capital cooperative nafed collaboration agriculture ministry established centre create awareness benefit millet encourage adoption among general public agriculture secretary manoj ahuja nafed md rajbir singh present event making effort enhance production yield processing consumption miller country tomar said minister highlighted multiple health benefit consuming millet nutritious tomar said millet climate resilient grown le water minimal use fertiliser pesticide said increase production millet also boost income farmer especially small marginal one tomar noted large number startup commendable job field millet rued importance millet plate food got reduced year acting upon india proposal supported country united nation general assembly declared

/Users/kusumachalla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/kusumachalla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
/Users/kusumachalla/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix

In [19]:
# len(doc_complete[0])
sentences = doc_complete[0].split(".")

In [21]:
len(sentences)

30

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_distances
import numpy as np
import string

# def preprocess_text(text):
#     # Convert text to lowercase
#     text = text.lower()

#     # Remove punctuation
#     text = text.translate(str.maketrans("", "", string.punctuation))

#     return text

def clustering_summarization(text, num_clusters, num_sentences):
    # Preprocess the text
#     text = preprocess_text(text)

    # Split the text into sentences using the delimiter "."
    sentences = text.split(".")

    # Remove empty sentences
    sentences = [sentence.strip() for sentence in sentences if sentence.strip() != ""]

    # Check the number of sentences
    if len(sentences) < num_clusters:
        num_clusters = len(sentences)

    # Vectorize the sentences using TF-IDF
    vectorizer = TfidfVectorizer()
    sentence_vectors = vectorizer.fit_transform(sentences)

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(sentence_vectors)

    # Initialize cluster representatives
    cluster_representatives = []

    # Select representative sentences from each cluster
    for cluster_id in range(num_clusters):
        cluster_sentences = [sentence for i, sentence in enumerate(sentences) if cluster_labels[i] == cluster_id]
        cluster_sentence_vectors = sentence_vectors[cluster_labels == cluster_id]

        # Calculate the centroid vector for the cluster
        centroid = np.mean(cluster_sentence_vectors, axis=0)

        # Calculate the cosine distances between each sentence vector in the cluster and the centroid
        distances = cosine_distances(cluster_sentence_vectors, centroid)

        # Sort sentences based on their distances to the centroid
        sorted_sentences = sorted(enumerate(cluster_sentences), key=lambda x: distances[x[0]], reverse=False)

        # Select the top representative sentence from the cluster
        representative_sentence = sorted_sentences[0][1]
        cluster_representatives.append(representative_sentence)

    # Select top 'num_sentences' representatives as the summary
    summary = cluster_representatives[:num_sentences]

    # Join the summary sentences into a single string
    summary_text = '. '.join(summary)

    return summary_text

generated_summaries = []
num_clusters = 4
num_sentences = 4

text = doc_complete[0]  # Select the first document from the list

for doc in doc_complete:

    summary = clustering_summarization(doc, num_clusters, num_sentences)
    print(summary)
    generated_summaries.append(summary)
    print("\n\n")
    print("--------------------------------------------------------------------------------------------------")
    print("\n\n")


The Government of India to encourage millets cultivation and consumption declared 2018 as the national year of millets. While this is already a vast number of people proponents of millet production are of the opinion that a greater number of people should include millet in their diet. We are the largest producer and the second largest exporter of Shri Anna Millets in the world. Millets have been a staple of the Indian diet especially in rural India for years and remain prevalent even today



--------------------------------------------------------------------------------------------------



The delegates will be taken to Sarnath on April 18. The district administration has also made elaborate arrangements for the delegates to facilitate their convenient movement in the city. The theme of the meeting is Sustainable Agriculture and Food Systems for Healthy People and the Planet. Thereafter a welcome dinner and cultural programme is organised at Taj Ganges



---------------------------

In [11]:
def extract_text_between_markers(text, start_marker, end_marker):
    extracted_texts = []
    start_index = 0

    while True:
        start_index = text.find(start_marker, start_index)
        if start_index == -1:
            break

        start_index += len(start_marker)
        end_index = text.find(end_marker, start_index)
        if end_index == -1:
            break

        extracted_text = text[start_index:end_index].strip()
        extracted_texts.append(extracted_text)

        start_index = end_index + len(end_marker)

    return extracted_texts

In [12]:
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

reference_summaries =[]
file_path = "News articles.docx"
text_content = read_docx(file_path)


start_marker = "Summary:"
end_marker = "Top Sentences:"

extracted_texts = extract_text_between_markers(text_content, start_marker, end_marker)
for extracted_text in extracted_texts:
    reference_summaries.append(extracted_text)


In [13]:
reference_summaries

["The Indian government, led by Agriculture Minister Narendra Singh Tomar, has opened the 'Millets Experience Centre' in New Delhi to promote the production and consumption of millets. The center aims to raise awareness about the nutritional benefits of millets and encourage their adoption among the public. Tomar emphasized the health advantages of millets and their climate resilience, highlighting that increased millet production would benefit farmers and startups in the sector. The United Nations has declared 2023 as the International Year of Millets, further positioning India as a leader in promoting this crop.",
 "India is taking initiatives to educate people about the nutritional value of millets and promote their consumption. Despite being rich in protein and antioxidants, millets have not been considered fashionable foods. However, the Indian government has recognized millets as a reliable grain for enhancing farmers' income and ensuring food security. Efforts to revive millet p

# Bleu Scores

In [20]:
from nltk.translate.bleu_score import sentence_bleu

bleu_score = sentence_bleu(generated_summaries[0],reference_summaries[0])
print("BLEU score for Article",":", bleu_score)


BLEU score for Article : 8.618388845839317e-232


In [21]:
bleu_score = sentence_bleu(generated_summaries[2],reference_summaries[1])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 8.436134752850363e-232


In [18]:
bleu_score = sentence_bleu(generated_summaries[3],reference_summaries[2])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 9.252921945181717e-232


In [22]:
bleu_score = sentence_bleu(generated_summaries[1],reference_summaries[3])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 8.24292407145299e-232


In [19]:
bleu_score = sentence_bleu(generated_summaries[4],reference_summaries[4])
print("BLEU score for Article",":", bleu_score)

BLEU score for Article : 9.338418756557488e-232
